In [1]:
import pandas as pd
import numpy as np 
import pickle 

corrMatrix = pickle.load(open('corrmtx.pkl','rb'))
model = pickle.load(open('model.pkl','rb'))
df = pd.read_csv('Users.csv')

In [2]:
customer_data = pd.read_csv('ColumnCountsperCustomer.csv')

In [3]:
def get_similar(itemId):
    similar_ratings = corrMatrix[itemId]
    index_list=similar_ratings.index
    value_list = similar_ratings.values
    final_df = pd.DataFrame(index=index_list)
    final_df['Corr'] = value_list
    final_df = final_df.sort_values(by='Corr',ascending=False)
    return final_df

In [4]:
def get_item_item(test_user) :
    user_items = []
    data = customer_data[customer_data['CustomerNumber']==test_user]
    user_items = [column for column in data.columns if data[column].iloc[0]!=0][1:]
    similar_items = pd.DataFrame()
    for item in user_items:
        recommended_df = get_similar(item)
        items = recommended_df.index
        for item in items:
            if item not in user_items:
                similar_items = similar_items.append(recommended_df.loc[item])
    similar_items = similar_items[similar_items['Corr']>0]
    return list(similar_items.index)
        

In [5]:
def get_item_users(test_user):
    distances,indices = model.kneighbors(df[df['Customer Code']==test_user][['DaysPresent','Revenue']],n_neighbors=6)
    indices = indices.squeeze()[1:]
    similar_users=[df['Customer Code'].values[i] for i in indices]
    user_items = []
    data = customer_data[customer_data['CustomerNumber']==test_user]
    user_items = [column for column in data.columns if data[column].iloc[0]!=0]
    recommended_items= []
    for user in similar_users :
        if user in list(customer_data['CustomerNumber']):
            data = customer_data[customer_data['CustomerNumber']==user]
#         print(data.head())
            for column in data.columns[1:] :
                if data[column].iloc[0]!=0:
                    if column not in user_items:
                        recommended_items.append(column)
    return recommended_items

In [6]:
item_item_recommendation = np.unique(list(get_item_item('CSH1965')))


In [7]:
user_item_recommendation = get_item_users('CSH1965')

In [8]:
final_recommendation = []
confidence = []
for item in user_item_recommendation:
    if item in item_item_recommendation:
        final_recommendation.append(item)
        confidence.append(1)
    else:
        final_recommendation.append(item)
        confidence.append(0)
        

In [9]:
recommendation_df = pd.DataFrame()
recommendation_df['Item'] = final_recommendation
recommendation_df['Confidence'] = confidence

recommendation_df[recommendation_df['Confidence']==1].shape

(25, 2)

In [10]:
%who

confidence	 corrMatrix	 customer_data	 final_recommendation	 get_item_item	 get_similar	 item	 item_item_recommendation	 np	 
pd	 pickle	 recommendation_df	 user_item_recommendation	 
